In [1]:
import numpy as np
import numpy.linalg as la
# import scipy.sparse as sparse

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

import timeit
import numba
from numba import jit

from iteration_methods import*
from basic import*

In [2]:
#output settings
np.set_printoptions(precision=2)
%matplotlib qt 

In [3]:
def u_analytical(r, th):

    # uan = np.sin(np.pi*r*np.cos(th))*np.sin(np.pi*r*np.sin(th)) + r*np.cos(th)

    # uan = (r+1)**(-1/3) + np.log(r+1) - np.cos(th) + np.exp(r*th)

    uan = r**3 + np.exp(th)

    return uan

#-------------------------------------------------------
#The source term of the equation
def source(x, y):

    # f = -2*np.pi**2*np.sin(np.pi*r*np.cos(th))*np.sin(np.pi*r*np.sin(th))

    # f = 4/9*(r+1)**(-7/3) - 1/(r+1)**2 + th**2*np.exp(r*th) + np.cos(th) + r**2*np.exp(r*th)

    f = 6*r + np.exp(th)

    return f

In [4]:
#Setup the parameters for the solver. Functions grid(), source() are in the basic.py module

R, TH, r, th, dr, dth = polar_grid(th_I=-np.pi, th_F=np.pi)                     #The grid of the problem

f = source(r, th)                                #source term

u_an = u_analytical(r, th)                       #analytical solution for comparison

u_guess = r**(-1/3) + np.log(2*r+1) - np.cos(th**2) + 5*np.exp(r*th+3)       #initial guess

# boundary = ((0,np.sin(np.pi*np.cos(TH))*np.sin(np.pi*np.sin(TH))+np.cos(TH)), (-R, -R))              #dirichlet boundary conditions
boundary = ((u_an[:,0], u_an[:,-1]), (u_an[0,:], u_an[-1,:]))

iterations = 20000                              #max number of iteration
tolerance = 1.e-6                               #desired tolerance

omega_opt = 2/(1+np.sin(np.pi*max(dr, dth)))     #relaxation parameter for SOR method

In [5]:
SOR_polar = jit(nopython=True)(SOR_polar)

In [6]:
#Calculate the results, time them and print output

start = timeit.default_timer()

u, k, rel_diff, conv_hist = SOR_polar(np.eye(99), f, (r, th), init_guess=u_guess, boundary=boundary,                                                                         tolerance=tolerance, itermax=iterations, omega=omega_opt)

elapsed = timeit.default_timer() - start

error_to_an = la.norm(u - u_an, 2)/la.norm(u_an, 2)

print('SOR solver \nNumber of iterations: {:} \nLast Relative difference: {:1.3E}' 
      '\nError to analytical: {:1.3E}s \nElapsed time: {:1.2E}s '.format(k, rel_diff, error_to_an, elapsed))
print('-'*50)

SOR solver 
Number of iterations: 2745 
Last Relative difference: 9.986E-07
Error to analytical: 4.489E-01s 
Elapsed time: 2.39E+00s 
--------------------------------------------------


In [7]:
#Plot results. Function plot is in basic.py module.
plot(r*np.cos(th), r*np.sin(th), u, u_an=u_an, conv_hist=conv_hist)


True
